### gift shop 지도 시각화

In [ ]:
import pandas as pd
import requests
from urllib.parse import quote

In [ ]:
# API 키 가져오기
with open('C:/workspace/kakao.txt') as file:
    road_key = file.read()

# URL 만들기
base_url = 'https://dapi.kakao.com/v2/local/search/address.json'
addr = '제주특별자치도 제주시 서해안로 626'
url = f'{base_url}?query={quote(addr)}'

# Header : Authorization: KakaoAK ${REST_API_KEY}
header = {'Authorization': f'KakaoAK {road_key}'}

# Kakao Local API 호출하여 결과 가져오기
result = requests.get(url, headers=header).json()
result

In [ ]:
lat = result['documents'][0]['y'] # 위도(latitude)
lng = result['documents'][0]['x'] # 경도(latitude)
dong=result['documents'][0]['address']['region_3depth_name']

In [ ]:
# 제주도 기념품샵 정보 완성하기
df = pd.read_csv("./제주도 기념품샵 목록.csv")
df

In [ ]:
lat_list, lng_list, dong_list = [], [], [] # 빈 리스트 생성. 위도와 경도 값 저장
for i in df.index: # df의 각 행에 대해 반복
    url = f'{base_url}?query={quote(df["주소"][i])}'
    result = requests.get(url, headers=header).json() # API에 요청을 보내고 응답을 JSON 형식으로 받음
    try:
        lat_list.append(result['documents'][0]['y']) # 위도(y) 추출하여 lat_list에 추가
        lng_list.append(result['documents'][0]['x']) # 경도(x) 추출하여 lng_list에 추가
        dong_list.append(result['documents'][0]['address']['region_3depth_name'])
    except:
        print(df.이름[i])  
# try-except 구문: try블록 실행 중 예외 발생 시 except블록 실행

In [ ]:
df[df['이름']=='선물고팡 제주공항점']

# map함수를 활용해 선물고팡 제주공항점 주소 변경
df.loc[203:321, '주소'] = df.loc[203:321]['주소'].map({'제주특별자치도 제주시 월성로 15 1,2층 ':'제주특별자치도 제주시 월성로 15 1'})

In [ ]:
df[df['이름']=='더아일랜더']

# 더아일랜더 주소 변경
df.loc[1172:1220, '주소'] = df.loc[1172:1220]['주소'].map({'제주특별자치도 제주시 관덕로4길 7 1,2층 ':'제주특별자치도 제주시 관덕로4길 7 1'})

In [ ]:
df[df['이름']=='제주고래감성소품']

# 제주고래감성소품 주소 변경
df.loc[1655:1656, '주소'] = df.loc[1655:1656]['주소'].map({'제주특별자치도 제주시 애월읍 애월해안로 390 1,2층 ':'제주특별자치도 제주시 애월읍 애월해안로 390 1'})

In [ ]:
# 수정된 데이터로 다시 위도경도 받아오기
lat_list, lng_list, dong_list = [], [], [] # 빈 리스트 생성. 위도와 경도 값 저장
for i in df.index: # df의 각 행에 대해 반복
    url = f'{base_url}?query={quote(df["주소"][i])}'
    result = requests.get(url, headers=header).json() # API에 요청을 보내고 응답을 JSON 형식으로 받음
    try:
        lat_list.append(result['documents'][0]['y']) # 위도(y) 추출하여 lat_list에 추가
        lng_list.append(result['documents'][0]['x']) # 경도(x) 추출하여 lng_list에 추가
        dong_list.append(result['documents'][0]['address']['region_3depth_name'])
    except:
        print(df.이름[i])
# try-except 구문: try블록 실행 중 예외 발생 시 except블록 실행

In [ ]:
# '선물고팡 제주공항점', '두몰', '더아일랜더', '제주고래감성소품', '제라몽' 행 제거
df=df[df.이름 != '선물고팡 제주공항점']
df=df[df.이름 != '도두몰']
df=df[df.이름 != '더아일랜더']
df=df[df.이름 != '제주고래감성소품']
df=df[df.이름 != '제라몽']

In [ ]:
df['위도']=lat_list
df['경도']=lng_list
df['주소2']=dong_list

# 제주도 기념품샵 위도 경도 동 리스트
lat_list
lng_list
dong_list

In [ ]:
df.to_csv("제주도 기념품샵 목록.csv", index=False)

In [ ]:
df = pd.read_csv('./제주도 기념품샵 목록.csv')      
df

In [ ]:
import folium


# folium.Map() 함수 사용하여 지도 객체 생성하기
# df의 위도와 경도 열의 평균값인 위치에 해당하는 지도를 생성하고 확대 수준을 10으로 설정
map=folium.Map(location=[df.위도.mean(), df.경도.mean()], zoom_start=10)
map

In [ ]:
# 제주시(blue)-서귀포시(green) 구분해서 folium.Marker
tiles = "http://mt0.google.com/vt/lyrs=m&hl=ko&x={x}&y={y}&z={z}" # 구글지도 타일 설정
attr= "Google" # 속성 설정
map = folium.Map(location=[df['위도'].mean(), df['경도'].mean()], # folium을 사용하여 지도 초기화
                 zoom_start=10, tiles=tiles, attr=attr)
for i in df.index: # df의 각 행을 반복
    if '제주시' in df.loc[i, '주소']:
        folium.Marker(
            location=[df.loc[i, '위도'], df.loc[i, '경도']],
            tooltip=df.loc[i, '이름'], # 마커에 마우스를 올렸을 때 표시되는 툴팁으로 이름 표시
            popup=folium.Popup(df.loc[i, '주소'], max_width=200),
            icon=folium.Icon(color='green', icon='glyphicon-gift')
        ).add_to(map)
    else:
        folium.Marker(
            location=[df.loc[i, '위도'], df.loc[i, '경도']],
            tooltip=df.loc[i, '이름'], # 마커에 마우스를 올렸을 때 표시되는 툴팁으로 이름 표시
            popup=folium.Popup(df.loc[i, '주소'], max_width=200),
            icon=folium.Icon(color='blue', icon='glyphicon-gift')
        ).add_to(map)

map        